# Model recovery attack: `wine`

In [1]:
%load_ext autoreload
%autoreload 2

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

import mra
from mra import *

os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

In [2]:
def make_dataset(X, Y, f):
    x = tf.data.Dataset.from_tensor_slices(X)
    y = tf.data.Dataset.from_tensor_slices(Y)
    x = x.map(f)
    xy = tf.data.Dataset.zip((x, y))
    xy = xy.shuffle(10000)
    return xy

xy = np.loadtxt("../datasets/wine.csv", delimiter=";")
np.random.shuffle(xy)
x = xy[:,0:-1]
y = xy[:,-1]

min_values = np.array([min(x[:,i]) for i in range(x.shape[1])])
max_values = np.array([max(x[:,i]) for i in range(x.shape[1])])
x = (x-min_values)/(max_values - min_values)
y = y.reshape((len(x), 1)).astype("float32")
train_ds = make_dataset(x, y, lambda t: t)

mra = mra(train_ds)

2022-02-16 00:47:02.005153: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-02-16 00:47:02.005196: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: xgpg2
2022-02-16 00:47:02.005202: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: xgpg2
2022-02-16 00:47:02.005286: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 470.103.1
2022-02-16 00:47:02.005305: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 470.103.1
2022-02-16 00:47:02.005310: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:310] kernel version seems to match DSO: 470.103.1
2022-02-16 00:47:02.005592: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in per

In [3]:
def make_f(input_shape):
    xin = tf.keras.layers.Input(input_shape)
    x = tf.keras.layers.Dense(64, activation="relu")(xin)
    x = tf.keras.layers.Dense(64, activation="relu")(x)
    x = tf.keras.layers.Dense(64, activation="relu")(x)
    output = tf.keras.layers.Dense(64, activation="relu")(x)
    return tf.keras.Model(xin, output)

def make_g(input_shape):
    xin = tf.keras.layers.Input(input_shape)
    # x = tf.keras.layers.Dense(64, activation="relu")(xin)
    # x = tf.keras.layers.Dropout(0.5)(x)
    output = tf.keras.layers.Dense(11, activation="softmax")(xin)
    return tf.keras.Model(xin, output)

In [4]:
log_train = mra.train(
    make_f,
    make_g,
    loss_fn=tf.keras.losses.sparse_categorical_crossentropy,
    batch_size=32,
    epoch=5,
    lr=0.001,
    verbose=True, log_every=50
    )

Iteration 0001: Training loss: 2.4393
Iteration 0051: Training loss: 1.0378
Iteration 0101: Training loss: 1.1548
Iteration 0151: Training loss: 0.7803
Iteration 0201: Training loss: 0.7742


In [5]:
def make_generator(input_shape):
    xin = tf.keras.layers.Input(input_shape)
    act = "relu"
    x = tf.keras.layers.Dense(512, activation=act)(xin)
    x = tf.keras.layers.Dense(256, activation=act)(x)
    x = tf.keras.layers.Dense(128, activation=act)(x)
    x = tf.keras.layers.Dense(64, activation=act)(x)
    x = tf.keras.layers.Dense(32, activation=act)(x)
    x = tf.keras.layers.Dense(11, activation="sigmoid")(x)
    return tf.keras.Model(xin, x)

In [6]:
log_attack = mra.attack(
    attack_iter=100,
    make_generator=make_generator,
    input_noise="normal",
    input_z=True,
    model_leak=True,
    lr_x=0.001,
    lr_f=0.0001,
    epoch=200,
    iter_x=50,
    iter_f=1,
    verbose=True, log_every=1)

Iteration 0001: RG-uniform: 0.1815, RG-normal: 0.1535, reconstruction validation: 0.0311
Iteration 0002: RG-uniform: 0.1857, RG-normal: 0.1479, reconstruction validation: 0.0217
Iteration 0003: RG-uniform: 0.1776, RG-normal: 0.1525, reconstruction validation: 0.0158
Iteration 0004: RG-uniform: 0.1564, RG-normal: 0.1526, reconstruction validation: 0.0098
Iteration 0005: RG-uniform: 0.1752, RG-normal: 0.1625, reconstruction validation: 0.0081
Iteration 0006: RG-uniform: 0.1595, RG-normal: 0.1465, reconstruction validation: 0.0073
Iteration 0007: RG-uniform: 0.1614, RG-normal: 0.1636, reconstruction validation: 0.0029
Iteration 0008: RG-uniform: 0.1371, RG-normal: 0.1480, reconstruction validation: 0.0096
Iteration 0009: RG-uniform: 0.1838, RG-normal: 0.1705, reconstruction validation: 0.0032
Iteration 0010: RG-uniform: 0.1753, RG-normal: 0.1530, reconstruction validation: 0.0057
Iteration 0011: RG-uniform: 0.1630, RG-normal: 0.1461, reconstruction validation: 0.0060
Iteration 0012: RG-un

In [7]:
np.savetxt("./log/wine-4.csv", log_attack, delimiter=",")